In [1]:
from openai import OpenAI

from langchain_openai import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.chains import RetrievalQA  
from langchain_pinecone import PineconeVectorStore

from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

import json

import pandas as pd
import os

In [2]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

chat_model = ChatOpenAI(api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)

In [3]:
client = OpenAI()

response = client.embeddings.create(
    input="hello",
    model="text-embedding-ada-002"
)

print(response.data[0].embedding)

[-0.025150660425424576, -0.019543994218111038, -0.027953993529081345, -0.03099534474313259, -0.024740738794207573, 0.027583742514252663, -0.012634835205972195, -0.008449669927358627, -0.017613396048545837, -0.008284378796815872, 0.03250279650092125, 0.004304174333810806, -0.024582060053944588, -0.0006231465958990157, 0.014135675504803658, -0.0016231564804911613, 0.03943179175257683, 0.0021504343021661043, 0.027041587978601456, -0.012489378452301025, -0.021091116592288017, 0.008965377695858479, 0.00839016493409872, -0.003259536111727357, -0.0053917886689305305, -0.00962654035538435, 0.011219944804906845, -0.0015396844828501344, 0.003557059681043029, -0.023286178708076477, 0.006783537566661835, -0.007854622788727283, -0.023907672613859177, -0.008912484161555767, 0.006862877402454615, -0.013778647407889366, 0.009533978067338467, -0.01420179195702076, 0.021990299224853516, -0.01057861652225256, 0.0033884630538523197, -0.014664607122540474, 0.00538187101483345, -0.014862955547869205, -0.031

In [18]:
df = pd.read_csv('./data/receipe_dataset_demo.csv')
df.head(1)

,DishName,CookingMethod,CookingOccasion,MainIngredients,CuisineType,DishDescription,IngredientDetails,ServingSize,DifficultyLevel,CookingTime
0,어묵김말이,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내


In [ ]:
# def process_ingredients(ingredient_str):
#     ingredient_str = ingredient_str.replace("[재료] ", "")
#     return [f"'{ingredient.strip()}'" for ingredient in ingredient_str.split('|')]

# df['Ingredients'] = df['IngredientDetails'].apply(process_ingredients)

In [46]:
proc = pd.read_csv('./data/whole_processed.csv')
proc.head()

,Recipe_ID,Recipe_Title,Dish_Name,Cooking_Method,Cooking_Status,Ingredient_List,Dish_Type,Cooking_Intro,Ingredient_Details,Serving_Size,Difficulty_Level,Cooking_Time
0,128671,어묵김말이,어묵김말이,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내
1,128892,두부에 꼬리가 달렸어요!!,두부새우전,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...,3인분,초급,30분이내
2,128932,입안에서 톡톡톡,알밥,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...,2인분,초급,30분이내
3,131871,★현미호두죽,현미호두죽,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내
4,139247,부들부들 보들보들 북어갈비♥,북어갈비,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,2인분,초급,60분이내


In [47]:
list_without_recipe_id = proc.drop(columns=['Recipe_ID']).apply(lambda row: ', '.join([f'{col}: {val}' for col, val in row.items()]), axis=1).tolist()
list_without_recipe_id

['Recipe_Title: 어묵김말이, Dish_Name: 어묵김말이, Cooking_Method: 튀김, Cooking_Status: 간식, Ingredient_List: 가공식품류, Dish_Type: 디저트, Cooking_Intro: 맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-, Ingredient_Details: [재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개| 깻잎 6장| 튀김가루 1컵 | 올리브유 적당량| 간장 1T| 참기름 1T, Serving_Size: 2인분, Difficulty_Level: 초급, Cooking_Time: 60분이내',
 'Recipe_Title: 두부에 꼬리가 달렸어요!!, Dish_Name: 두부새우전, Cooking_Method: 부침, Cooking_Status: 일상, Ingredient_List: 해물류, Dish_Type: 밑반찬, Cooking_Intro: 꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부새우전. 모양도 이쁘고 맛까지 좋은 두부새우전!! 함께 만들어 보아요♥, Ingredient_Details: [재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마리| 녹말가루| 계란 1개, Serving_Size: 3인분, Difficulty_Level: 초급, Cooking_Time: 30분이내',
 'Recipe_Title: 입안에서 톡톡톡, Dish_Name: 알밥, Cooking_Method: 굽기, Cooking_Status: 일상, Ingredient_List: 해물류, Dish_Type: 밥/죽/떡, Cooking_Intro: 간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠~ 색깔도 너무 이쁘고 한 그릇만 있으면 반찬도 필요없는 알밥, Ingredient_Details: [재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무순 약간|

In [48]:
embeddings = []
for i in list_without_recipe_id:
    response = client.embeddings.create(
        input=i,
        model="text-embedding-ada-002"
    )
    embeddings.append(response.data[0].embedding)

embeddings

KeyboardInterrupt: 

In [ ]:
ids = proc['Recipe_ID'].to_list()
ids

[128671,
 128892,
 128932,
 131871,
 139247,
 149207,
 151148,
 153040,
 221094,
 221097]

In [37]:
meta_datas = [{'text': text} for text in list_without_recipe_id]
meta_datas

[{'text': 'Recipe_Title: 어묵김말이, Dish_Name: 어묵김말이, Cooking_Method: 튀김, Cooking_Status: 간식, Ingredient_List: 가공식품류, Dish_Type: 디저트, Cooking_Intro: 맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-, Ingredient_Details: [재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개| 깻잎 6장| 튀김가루 1컵 | 올리브유 적당량| 간장 1T| 참기름 1T, Serving_Size: 2인분, Difficulty_Level: 초급, Cooking_Time: 60분이내'},
 {'text': 'Recipe_Title: 두부에 꼬리가 달렸어요!!, Dish_Name: 두부새우전, Cooking_Method: 부침, Cooking_Status: 일상, Ingredient_List: 해물류, Dish_Type: 밑반찬, Cooking_Intro: 꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부새우전. 모양도 이쁘고 맛까지 좋은 두부새우전!! 함께 만들어 보아요♥, Ingredient_Details: [재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마리| 녹말가루| 계란 1개, Serving_Size: 3인분, Difficulty_Level: 초급, Cooking_Time: 30분이내'},
 {'text': 'Recipe_Title: 입안에서 톡톡톡, Dish_Name: 알밥, Cooking_Method: 굽기, Cooking_Status: 일상, Ingredient_List: 해물류, Dish_Type: 밥/죽/떡, Cooking_Intro: 간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠~ 색깔도 너무 이쁘고 한 그릇만 있으면 반찬도 필요없는 알밥, Ingredient_Details: [재료] 밥 1+1/2공기| 당근 1/4

In [38]:
records = [{'id': str(i), 'values': j, 'metadata': k} for i,j,k in zip(ids, embeddings, meta_datas)]
records

[{'id': '128671',
  'values': [-0.005670077167451382,
   -0.0004469765117391944,
   0.015055367723107338,
   -0.02948829159140587,
   -0.007599005941301584,
   0.010263844393193722,
   -0.024301256984472275,
   -0.03226335719227791,
   -0.03929178789258003,
   -0.023173077031970024,
   -0.0066718230955302715,
   0.01863442175090313,
   -0.03493467718362808,
   -0.006156361661851406,
   -0.027491284534335136,
   0.025351632386446,
   0.030525699257850647,
   0.0054561118595302105,
   -0.005634416360408068,
   -0.014510729350149632,
   -0.0012740653473883867,
   -0.0013461975613608956,
   0.00436683464795351,
   0.003912969026714563,
   -0.0002820450172293931,
   0.03620550036430359,
   -0.00025023389025591314,
   -0.01633915863931179,
   -0.02428828924894333,
   0.02118903584778309,
   -6.448334897868335e-05,
   -0.02225237898528576,
   -0.008539156056940556,
   -0.015236914157867432,
   -0.018647389486432076,
   0.02008679136633873,
   0.01727282628417015,
   0.00012471171794459224,
  

In [25]:
index = pc.Index('receipe')

In [ ]:
index.delete(delete_all=True)

In [39]:
index.upsert(vectors=records)

upserted_count: 10

In [27]:
def request_query(query):
    response = client.embeddings.create(
        input=query,
        model="text-embedding-ada-002"
    )

    embedded_query=response.data[0].embedding
    results=index.query(embedded_query, top_k=3, include_metadata=True)

    return results['matches']

In [42]:
pinecone_result = request_query('무더운 날 먹기 좋은 음식, Ingredient_Details: 어묵 2개| 김밥용김 3장| 당면 1웅큼')

In [43]:
pinecone_result

[{'id': '128671',
  'metadata': {'text': 'Recipe_Title: 어묵김말이, Dish_Name: 어묵김말이, Cooking_Method: '
                       '튀김, Cooking_Status: 간식, Ingredient_List: 가공식품류, '
                       'Dish_Type: 디저트, Cooking_Intro: 맛있는 김말이에 쫄깃함을 더한 어묵 '
                       '김말이예요-, Ingredient_Details: [재료] 어묵 2개| 김밥용김 3장| 당면 '
                       '1움큼| 양파 1/2개| 당근 1/2개| 깻잎 6장| 튀김가루 1컵 | 올리브유 적당량| 간장 '
                       '1T| 참기름 1T, Serving_Size: 2인분, Difficulty_Level: 초급, '
                       'Cooking_Time: 60분이내'},
  'score': 0.8987191,
  'sparse_values': {'indices': [], 'values': []},
  'values': []},
 {'id': '128892',
  'metadata': {'text': 'Recipe_Title: 두부에 꼬리가 달렸어요!!, Dish_Name: 두부새우전, '
                       'Cooking_Method: 부침, Cooking_Status: 일상, '
                       'Ingredient_List: 해물류, Dish_Type: 밑반찬, Cooking_Intro: '
                       '꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부새우전. 모양도 '
                       '이쁘고 맛까지 좋은 두부새우전!! 함께 만들어 보아요♥, Ingr

In [53]:
df.iloc[1,2]

["'두부 1/2모'",
 "'당근 1/2개'",
 "'고추 2개'",
 "'브로콜리 1/4개'",
 "'새우 4마리'",
 "'녹말가루'",
 "'계란 1개'"]

In [44]:
raw = pd.read_csv('./data/whole_raw.csv')
raw.columns = [
    "Recipe_ID",
    "Recipe_Title",
    "Dish_Name",
    "Creator_ID",
    "Creator_Name",
    "View_Count",
    "Recommendation_Count",
    "Scrap_Count",
    "Cooking_Method",
    "Cooking_Status",
    "Ingredient_List",
    "Dish_Type",
    "Cooking_Intro",
    "Ingredient_Details",
    "Serving_Size",
    "Difficulty_Level",
    "Cooking_Time",
    "Initial_Registration_Date"
]

raw = raw[['Recipe_ID', 'Recipe_Title', 'Dish_Name', 'Cooking_Method', 'Cooking_Status', 'Ingredient_List',
           'Dish_Type', 'Cooking_Intro', 'Ingredient_Details', 'Serving_Size', 'Difficulty_Level',
           'Cooking_Time']]
raw.head(1)

,Recipe_ID,Recipe_Title,Dish_Name,Cooking_Method,Cooking_Status,Ingredient_List,Dish_Type,Cooking_Intro,Ingredient_Details,Serving_Size,Difficulty_Level,Cooking_Time
0,128671,어묵김말이,어묵김말이,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내


In [45]:
raw.to_csv('./data/whole_processed.csv', index=False)

In [101]:
proc = pd.read_csv('./data/processed.csv')
proc.head()

,Recipe_ID,Recipe_Title,Dish_Name,Cooking_Method,Cooking_Status,Ingredient_List,Dish_Type,Cooking_Intro,Ingredient_Details,Serving_Size,Difficulty_Level,Cooking_Time
0,128671,어묵김말이,어묵김말이,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내
1,128892,두부에 꼬리가 달렸어요!!,두부새우전,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...,3인분,초급,30분이내
2,128932,입안에서 톡톡톡,알밥,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...,2인분,초급,30분이내
3,131871,★현미호두죽,현미호두죽,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내
4,139247,부들부들 보들보들 북어갈비♥,북어갈비,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,2인분,초급,60분이내


In [102]:
list_without_recipe_id = proc.drop(columns=['Recipe_ID']).apply(lambda row: ', '.join([f'{col}: {val}' for col, val in row.items()]), axis=1).tolist()
list_without_recipe_id

['Recipe_Title: 어묵김말이, Dish_Name: 어묵김말이, Cooking_Method: 튀김, Cooking_Status: 간식, Ingredient_List: 가공식품류, Dish_Type: 디저트, Cooking_Intro: 맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-, Ingredient_Details: [재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개| 깻잎 6장| 튀김가루 1컵 | 올리브유 적당량| 간장 1T| 참기름 1T, Serving_Size: 2인분, Difficulty_Level: 초급, Cooking_Time: 60분이내',
 'Recipe_Title: 두부에 꼬리가 달렸어요!!, Dish_Name: 두부새우전, Cooking_Method: 부침, Cooking_Status: 일상, Ingredient_List: 해물류, Dish_Type: 밑반찬, Cooking_Intro: 꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부새우전. 모양도 이쁘고 맛까지 좋은 두부새우전!! 함께 만들어 보아요♥, Ingredient_Details: [재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마리| 녹말가루| 계란 1개, Serving_Size: 3인분, Difficulty_Level: 초급, Cooking_Time: 30분이내',
 'Recipe_Title: 입안에서 톡톡톡, Dish_Name: 알밥, Cooking_Method: 굽기, Cooking_Status: 일상, Ingredient_List: 해물류, Dish_Type: 밥/죽/떡, Cooking_Intro: 간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠~ 색깔도 너무 이쁘고 한 그릇만 있으면 반찬도 필요없는 알밥, Ingredient_Details: [재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무순 약간|

In [86]:
summary = []
for i in range(0, len(proc)):
    summary.append(str(proc.iloc[i, 1:]))

summary

['Recipe_Title                                                      어묵김말이\nDish_Name                                                         어묵김말이\nCooking_Method                                                       튀김\nCooking_Status                                                       간식\nIngredient_List                                                   가공식품류\nDish_Type                                                           디저트\nCooking_Intro                                맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-\nIngredient_Details    [재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...\nServing_Size                                                        2인분\nDifficulty_Level                                                     초급\nCooking_Time                                                      60분이내\nName: 0, dtype: object',
 'Recipe_Title                                             두부에 꼬리가 달렸어요!!\nDish_Name                                                         두부새우전\nCooking_Method        

In [97]:
list_without_recipe_id = proc.drop(columns=['Recipe_ID']).apply(lambda row: ', '.join([f'{col}: {val}' for col, val in row.items()]), axis=1).tolist()
list_without_recipe_id

['Recipe_Title: 어묵김말이, Dish_Name: 어묵김말이, Cooking_Method: 튀김, Cooking_Status: 간식, Ingredient_List: 가공식품류, Dish_Type: 디저트, Cooking_Intro: 맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-, Ingredient_Details: [재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개| 깻잎 6장| 튀김가루 1컵 | 올리브유 적당량| 간장 1T| 참기름 1T, Serving_Size: 2인분, Difficulty_Level: 초급, Cooking_Time: 60분이내',
 'Recipe_Title: 두부에 꼬리가 달렸어요!!, Dish_Name: 두부새우전, Cooking_Method: 부침, Cooking_Status: 일상, Ingredient_List: 해물류, Dish_Type: 밑반찬, Cooking_Intro: 꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부새우전. 모양도 이쁘고 맛까지 좋은 두부새우전!! 함께 만들어 보아요♥, Ingredient_Details: [재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마리| 녹말가루| 계란 1개, Serving_Size: 3인분, Difficulty_Level: 초급, Cooking_Time: 30분이내',
 'Recipe_Title: 입안에서 톡톡톡, Dish_Name: 알밥, Cooking_Method: 굽기, Cooking_Status: 일상, Ingredient_List: 해물류, Dish_Type: 밥/죽/떡, Cooking_Intro: 간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠~ 색깔도 너무 이쁘고 한 그릇만 있으면 반찬도 필요없는 알밥, Ingredient_Details: [재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무순 약간|

In [94]:
list_with_columns = proc.apply(lambda row: ', '.join([f'{col}: {val}' for col, val in row.items()]), axis=1).tolist()
list_with_columns

KeyError: "['Recipe_ID'] not found in axis"

In [69]:
raw['summary'] = raw.apply(lambda row: f"레시피제목: {raw['RCP_TTL']}, 요리명: {raw['CKG_NM']}, "
                                     f"요리방법명: {raw['CKG_MTH_ACTO_NM']}, 요리상황명: {raw['CKG_STA_ACTO_NM']}, "
                                     f"요리재료명: {raw['CKG_MTRL_ACTO_NM']}, 요리종류명: {raw['CKG_KND_ACTO_NM']}, "
                                     f"요리소개: {raw['CKG_IPDC']}, 요리재료내용: {raw['CKG_MTRL_CN']}, "
                                     f"요리인분명: {raw['CKG_INBUN_NM']}, 요리난이도명: {raw['CKG_DODF_NM']}, "
                                     f"요리시간명: {raw['CKG_TIME_NM']}", axis=1)
raw.summary[1]

'레시피제목: 0                  어묵김말이\n1         두부에 꼬리가 달렸어요!!\n2               입안에서 톡톡톡\n3                 ★현미호두죽\n4        부들부들 보들보들 북어갈비♥\n5               토마토스파게티♥\n6                 표고버섯탕수\n7    달콤한 마늘향이 가득해~ 갈릭치킨♥\n8        바삭바삭 양파링 튀김 만들기\n9            참치 삼각김밥 삼총사\nName: RCP_TTL, dtype: object, 요리명: 0      어묵김말이\n1      두부새우전\n2         알밥\n3      현미호두죽\n4       북어갈비\n5    토마토스파게티\n6     표고버섯탕수\n7       갈릭치킨\n8      양파링튀김\n9     참치삼각김밥\nName: CKG_NM, dtype: object, 요리방법명: 0     튀김\n1     부침\n2     굽기\n3    끓이기\n4     굽기\n5     볶음\n6     튀김\n7     튀김\n8     튀김\n9     기타\nName: CKG_MTH_ACTO_NM, dtype: object, 요리상황명: 0      간식\n1      일상\n2      일상\n3      일상\n4     술안주\n5      일상\n6    손님접대\n7    손님접대\n8      간식\n9     도시락\nName: CKG_STA_ACTO_NM, dtype: object, 요리재료명: 0    가공식품류\n1      해물류\n2      해물류\n3        쌀\n4     건어물류\n5    가공식품류\n6      버섯류\n7      닭고기\n8      채소류\n9    가공식품류\nName: CKG_MTRL_ACTO_NM, dtype: object, 요리종류명: 0      디저트\n1      밑반찬\n2    밥/죽/떡\n3    밥/죽/떡\n4     메

In [50]:
raw.columns = [
    "레시피제목",
    "요리명",
    "요리방법명",
    "Creator_ID",
    "Creator_Name",
    "View_Count",
    "Recommendation_Count",
    "Scrap_Count",
    "Cooking_Method",
    "Cooking_Status",
    "Ingredient_List",
    "Dish_Type",
    "Cooking_Intro",
    "Ingredient_Details",
    "Serving_Size",
    "Difficulty_Level",
    "Cooking_Time",
    "Initial_Registration_Date"]
raw.head(1)

,Recipe_ID,Recipe_Title,Dish_Name,Creator_ID,Creator_Name,View_Count,Recommendation_Count,Scrap_Count,Cooking_Method,Cooking_Status,Ingredient_List,Dish_Type,Cooking_Intro,Ingredient_Details,Serving_Size,Difficulty_Level,Cooking_Time,Initial_Registration_Date
0,128671,어묵김말이,어묵김말이,skfo0701,꽃날,10072,6,66,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내,20070402131403


In [51]:
raw = raw[['Recipe_ID', 'Recipe_Title', 'Dish_Name', 'Cooking_Method', 'Cooking_Status', 'Ingredient_List',
           'Dish_Type', 'Cooking_Intro', 'Ingredient_Details', 'Serving_Size', 'Difficulty_Level',
           'Cooking_Time']]
raw.head()

,Recipe_ID,Recipe_Title,Dish_Name,Cooking_Method,Cooking_Status,Ingredient_List,Dish_Type,Cooking_Intro,Ingredient_Details,Serving_Size,Difficulty_Level,Cooking_Time
0,128671,어묵김말이,어묵김말이,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내
1,128892,두부에 꼬리가 달렸어요!!,두부새우전,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...,3인분,초급,30분이내
2,128932,입안에서 톡톡톡,알밥,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...,2인분,초급,30분이내
3,131871,★현미호두죽,현미호두죽,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내
4,139247,부들부들 보들보들 북어갈비♥,북어갈비,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,2인분,초급,60분이내


In [52]:
raw.to_csv('./data/processed.csv', index=False)

In [52]:
index_name = "receipe_ingredients"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=2,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        )
    )

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '9d91e97595e9e02d583379f42f2e72ec', 'Date': 'Thu, 15 Aug 2024 05:02:17 GMT', 'Server': 'Google Frontend', 'Content-Length': '125', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"INVALID_ARGUMENT","message":"Name must consist of lower case alphanumeric characters or '-'"},"status":400}


In [45]:
index = pc.Index(index_name)

index.upsert(
    vectors=[
        {"id": "vec1", "values": [1.0, 1.5]},
        {"id": "vec2", "values": [2.0, 1.0]},
        {"id": "vec3", "values": [0.1, 3.0]},
    ],
    namespace="ns1"
)

index.upsert(
    vectors=[
        {"id": "vec1", "values": [1.0, -2.5]},
        {"id": "vec2", "values": [3.0, -2.0]},
        {"id": "vec3", "values": [0.5, -1.5]},
    ],
    namespace="ns2"
)

upserted_count: 3

In [46]:
print(index.describe_index_stats())

{'dimension': 2,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 3}, 'ns2': {'vector_count': 3}},
 'total_vector_count': 6}


In [47]:
query_results1 = index.query(
    namespace="ns1",
    vector=[1.0, 1.5],
    top_k=3,
    include_values=True
)

print(query_results1)

query_results2 = index.query(
    namespace="ns2",
    vector=[1.0,-2.5],
    top_k=3,
    include_values=True
)

print(query_results2)

{'matches': [{'id': 'vec1',
              'metadata': None,
              'score': 1.0,
              'sparse_values': {'indices': [], 'values': []},
              'values': [1.0, 1.5]},
             {'id': 'vec2',
              'metadata': None,
              'score': 0.86824316,
              'sparse_values': {'indices': [], 'values': []},
              'values': [2.0, 1.0]},
             {'id': 'vec3',
              'metadata': None,
              'score': 0.85006815,
              'sparse_values': {'indices': [], 'values': []},
              'values': [0.1, 3.0]}],
 'namespace': 'ns1',
 'usage': {'read_units': 6}}
{'matches': [{'id': 'vec1',
              'metadata': None,
              'score': 1.0,
              'sparse_values': {'indices': [], 'values': []},
              'values': [1.0, -2.5]},
             {'id': 'vec3',
              'metadata': None,
              'score': 0.9982744,
              'sparse_values': {'indices': [], 'values': []},
              'values': [0.5, 

In [48]:
pc.delete_index(index_name)

In [ ]:
openai.api_key = os.getenv('OPENAI_API_KEY') or 'OPENAI_API_KEY'

embed_model = "text-embedding-ada-002"

In [ ]:
query = "how do I use the LLMChain in LangChain?"

res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(vector=xq, top_k=5, include_metadata=True)

In [60]:
from openai import OpenAI
client = OpenAI()

# response = client.embeddings.create(
#     input="Your text string goes here",
#     model="text-embedding-3-small"
# )

# print(response.data[0].embedding)

In [ ]:
openai.api_key = OPENAI_API_KEY
openai.Embedding.create(
  model="text-embedding-ada-002",
  input="The food was delicious and the waiter..."
)

In [ ]:
# 이름, 시간, 요리법, 재료, 추가필요재료